<a href="https://colab.research.google.com/github/IlyaZutler/Bus_lanes/blob/main/DM%20_%20LinesGrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np

In [62]:
# Assuming 'trips.xlsx' is uploaded to your Colab environment
trips = pd.read_excel('trips.xlsx', header=0)  # header=0 means the first row is the header

trips.sample(2)

,Car_ID,Day_time,Сoordinates,Speed,Ignition
1263,77403 (М947ОК67) Renault Logan,17.02.2025 18:10:18,"55,743215°, 37,821318°",22,+
1467,77403 (М947ОК67) Renault Logan,17.02.2025 20:38:57,"55,774371°, 37,826123°",29,+


In [63]:
# Split the 'Coordinates' column into 'Longitude' and 'Latitude'
trips['Сoordinates'] = trips['Сoordinates'].str.replace('°', '').str.replace(',', '.')
trips[['Longitude', 'Latitude']] = trips['Сoordinates'].str.split('. ', expand=True)
trips.drop(['Сoordinates'], axis=1, inplace=True)

# Convert 'Day_time' to datetime objects
trips['Day_time'] = pd.to_datetime(trips['Day_time'], errors='coerce')

trips['Ignition'] = trips['Ignition'].str.replace('-', '0').str.replace('+', '1')

trips

<ipython-input-63-fdec02703b05>:7: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  trips['Day_time'] = pd.to_datetime(trips['Day_time'], errors='coerce')


,Car_ID,Day_time,Speed,Ignition,Longitude,Latitude
0,77403 (М947ОК67) Renault Logan,2025-02-17 00:02:39,0,0,55.771763,37.83387
1,77403 (М947ОК67) Renault Logan,2025-02-17 00:07:39,0,0,55.771763,37.83387
2,77403 (М947ОК67) Renault Logan,2025-02-17 00:12:39,0,0,55.771763,37.83387
3,77403 (М947ОК67) Renault Logan,2025-02-17 00:17:39,0,0,55.771763,37.83387
4,77403 (М947ОК67) Renault Logan,2025-02-17 00:22:39,0,0,55.771763,37.83387
...,...,...,...,...,...,...
3606,77403 (М947ОК67) Renault Logan,2025-02-18 23:38:42,0,0,55.771658,37.833611
3607,77403 (М947ОК67) Renault Logan,2025-02-18 23:43:42,0,0,55.771658,37.833611
3608,77403 (М947ОК67) Renault Logan,2025-02-18 23:48:42,0,0,55.771658,37.833611
3609,77403 (М947ОК67) Renault Logan,2025-02-18 23:53:42,0,0,55.771658,37.833611


In [44]:
# Загрузка данных о Выделенных полосах
bus_lanes = gpd.read_file('bus-lanes.geojson')

In [54]:
print(bus_lanes['geometry'][2])

LINESTRING (49.0701771 55.8007985, 49.0743828 55.803681, 49.0747047 55.8042116)


In [45]:
# Преобразование поездок в геометрию
trips['geometry'] = trips.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
trips_gdf = gpd.GeoDataFrame(trips, geometry='geometry')

In [47]:
# Функция для поиска пересечений траектории с полосами - считает не то, что нужно.
def find_overlap(trajectory, bus_lane):
    buffer = bus_lane.buffer(0.0005)  # ~50 метров в десятичных градусах
    overlap = trajectory.within(buffer)
    return overlap

In [ ]:
# Группировка по машинам и построение траекторий
for car_id, group in trips_gdf.groupby('Car_ID'):
    group = group.sort_values('Day_time')
    trajectory = LineString(group['geometry'].tolist())

    # Проверка пересечения с каждой выделенной полосой
    for idx, lane in bus_lanes.iterrows():

        if find_overlap(trajectory, lane['geometry']): # если есть пересечение траектории машины и полосы
            # Вычисление времени и длины сегмента
            segment = group[group.within(lane['geometry'].buffer(0.0005))]
            time_current = (pd.to_datetime(segment['Day_time'].iloc[-1]) -
                           pd.to_datetime(segment['Day_time'].iloc[0])).total_seconds() / 3600
            length = lane['length'] / 1000  # в км
            speed_bus_lane = 50  # км/ч (пример)
            time_bus_lane = length / speed_bus_lane
            time_gain = time_current - time_bus_lane
            print(f"Машина {car_id}, сегмент {idx}: выигрыш {time_gain:.2f} часов")